**feature extraction✅ without augmentation**

#Import Data - Library

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# List files in the directory to verify existence
!ls /content/drive/MyDrive/TI14_Personal/mbti_1.csv

/content/drive/MyDrive/TI14_Personal/mbti_1.csv


In [ ]:
# List files in the directory to verify existence
!ls /content/drive/MyDrive/TI14_Personal/preprocessed_mbti1.csv

/content/drive/MyDrive/TI14_Personal/preprocessed_mbti1.csv


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from gensim.models import KeyedVectors
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from sklearn.preprocessing import StandardScaler

import nltk
import re
import os
import random
from collections import Counter
from google.colab import files
from nltk import pos_tag, ne_chunk
from nltk.util import ngrams

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
df_ori = pd.read_csv('/content/drive/MyDrive/TI14_Personal/mbti_1.csv')
#original dataset

In [ ]:
df_ori.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TI14_Personal/preprocessed_mbti1.csv')
#preprocessed mbti

In [ ]:
df.head()

,type,posts,posts_cleaned,I/E,N/S,F/T,J/P,tokens
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,moment sportscenter top ten play prank life ch...,I,N,F,J,"['moment', 'sportscenter', 'top', 'ten', 'play..."
1,ENTP,'I'm finding the lack of me in these posts ver...,finding lack post alarming sex boring position...,E,N,T,P,"['finding', 'lack', 'post', 'alarming', 'sex',..."
2,INTP,'Good one _____ https://www.youtube.com/wat...,good one course say know blessing curse absolu...,I,N,T,P,"['good', 'one', 'course', 'say', 'know', 'bles..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",dear enjoyed conversation day esoteric gabbing...,I,N,T,J,"['dear', 'enjoyed', 'conversation', 'day', 'es..."
4,ENTJ,'You're fired.|||That's another silly misconce...,fired another silly misconception approaching ...,E,N,T,J,"['fired', 'another', 'silly', 'misconception',..."


#Split Data

In [ ]:
# Check distribution before splitting
print("Distribution before splitting:")
print(df['type'].value_counts(normalize=True))

Distribution before splitting:
type
INFP    0.211182
INFJ    0.169452
INTP    0.150317
INTJ    0.125764
ENTP    0.078963
ENFP    0.077810
ISTP    0.038847
ISFP    0.031239
ENTJ    0.026628
ISTJ    0.023631
ENFJ    0.021902
ISFJ    0.019135
ESTP    0.010259
ESFP    0.005533
ESFJ    0.004841
ESTJ    0.004496
Name: proportion, dtype: float64


In [ ]:
# Split data into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['type'], random_state=42)

In [ ]:
# Check distribution after splitting
print("\nDistribution in training set:")
print(train_df['type'].value_counts(normalize=True))

print("\nDistribution in test set:")
print(test_df['type'].value_counts(normalize=True))


Distribution in training set:
type
INFP    0.211239
INFJ    0.169452
INTP    0.150288
INTJ    0.125793
ENTP    0.078963
ENFP    0.077810
ISTP    0.038905
ISFP    0.031268
ENTJ    0.026657
ISTJ    0.023631
ENFJ    0.021902
ISFJ    0.019164
ESTP    0.010231
ESFP    0.005476
ESFJ    0.004755
ESTJ    0.004467
Name: proportion, dtype: float64

Distribution in test set:
type
INFP    0.210951
INFJ    0.169452
INTP    0.150432
INTJ    0.125648
ENTP    0.078963
ENFP    0.077810
ISTP    0.038617
ISFP    0.031124
ENTJ    0.026513
ISTJ    0.023631
ENFJ    0.021902
ISFJ    0.019020
ESTP    0.010375
ESFP    0.005764
ESFJ    0.005187
ESTJ    0.004611
Name: proportion, dtype: float64


In [ ]:
# Save train and test sets to CSV
train_df.to_csv('train_set2.csv', index=False)
test_df.to_csv('test_set2.csv', index=False)

In [ ]:
# Download the file to your local machine
files.download('test_set2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load the CSV file into a new DataFrame
train_df = pd.read_csv('train_set2.csv')
test_df = pd.read_csv('test_set2.csv')

#Feature Extraction

In [ ]:
# Convert all entries in the 'posts_cleaned' column to strings, replacing NaNs with empty strings
train_df['posts_cleaned'] = train_df['posts_cleaned'].astype(str).fillna('')
test_df['posts_cleaned'] = test_df['posts_cleaned'].astype(str).fillna('')

In [ ]:
# Tokenize the data
train_tokens = train_df['posts_cleaned'].apply(word_tokenize)
test_tokens = test_df['posts_cleaned'].apply(word_tokenize)

In [ ]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=train_tokens, vector_size=400, window=5, min_count=1, workers=4)

In [ ]:
# Function to get Word2Vec features
def get_word2vec_features(text, model, num_features):
    words = text.split()
    feature_vector = np.zeros((num_features,), dtype="float32")
    for word in words:
        if word in model.wv:
            feature_vector = np.add(feature_vector, model.wv[word])
    if len(words) > 0:
        feature_vector = np.divide(feature_vector, len(words))
    return feature_vector

In [ ]:
# Extract Word2Vec features
train_word2vec_features = np.array([get_word2vec_features(text, word2vec_model, 400) for text in train_df['posts_cleaned']])
test_word2vec_features = np.array([get_word2vec_features(text, word2vec_model, 400) for text in test_df['posts_cleaned']])

In [ ]:
# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=400)
train_tfidf_features = tfidf_vectorizer.fit_transform(train_df['posts_cleaned']).toarray()
test_tfidf_features = tfidf_vectorizer.transform(test_df['posts_cleaned']).toarray()

In [ ]:
# Combine Word2Vec and TF-IDF features
train_combined_features = np.hstack((train_word2vec_features, train_tfidf_features))
test_combined_features = np.hstack((test_word2vec_features, test_tfidf_features))

#Model

In [ ]:
# Define a mapping from MBTI type to dimensions
def get_mbti_dimensions(mbti_type):
    ie = 1 if mbti_type[0] == 'I' else 0
    ns = 1 if mbti_type[1] == 'N' else 0
    ft = 1 if mbti_type[2] == 'F' else 0
    jp = 1 if mbti_type[3] == 'J' else 0
    return ie, ns, ft, jp

# Apply the function to both train and test sets
train_df[['I/E', 'N/S', 'F/T', 'J/P']] = train_df['type'].apply(lambda x: pd.Series(get_mbti_dimensions(x)))
test_df[['I/E', 'N/S', 'F/T', 'J/P']] = test_df['type'].apply(lambda x: pd.Series(get_mbti_dimensions(x)))

In [ ]:
# Define the RandomForest model
rf_model = RandomForestClassifier(random_state=42, max_depth=10)

In [ ]:
# Train the model on class type
rf_model.fit(train_combined_features, train_df['type'])

RandomForestClassifier(max_depth=10, random_state=42)

In [ ]:
# Predict on the train set and map to dimensions
train_predictions = rf_model.predict(train_combined_features)
train_pred_dimensions = pd.DataFrame([get_mbti_dimensions(pred) for pred in train_predictions], columns=['I/E', 'N/S', 'F/T', 'J/P'])

In [ ]:
# Predict on the test set and map to dimensions
test_predictions = rf_model.predict(test_combined_features)
test_pred_dimensions = pd.DataFrame([get_mbti_dimensions(pred) for pred in test_predictions], columns=['I/E', 'N/S', 'F/T', 'J/P'])

In [ ]:
# Initialize lists to store metrics for each dimension
average_accuracies = []
precisions = []
recalls = []
f1_scores = []

# Calculate metrics for each dimension and their averages
for dimension in ['I/E', 'N/S', 'F/T', 'J/P']:
    train_accuracy = accuracy_score(train_df[dimension], train_pred_dimensions[dimension])
    test_accuracy = accuracy_score(test_df[dimension], test_pred_dimensions[dimension])

    # Calculate the average accuracy for this dimension
    average_accuracy = (train_accuracy + test_accuracy) / 2
    average_accuracies.append(average_accuracy)

    # Calculate precision, recall, and f1-score for the test set
    precision = precision_score(test_df[dimension], test_pred_dimensions[dimension])
    recall = recall_score(test_df[dimension], test_pred_dimensions[dimension])
    f1 = f1_score(test_df[dimension], test_pred_dimensions[dimension])

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    print(f"Train Accuracy for {dimension}: {train_accuracy}")
    print(f"Test Accuracy for {dimension}: {test_accuracy}")
    print(f"Average Accuracy for {dimension}: {average_accuracy}")
    print(f"Precision for {dimension}: {precision}")
    print(f"Recall for {dimension}: {recall}")
    print(f"F1-Score for {dimension}: {f1}\n")

# If you want to calculate an overall average accuracy, precision, recall, and f1-score across all dimensions
overall_average_accuracy = sum(average_accuracies) / len(average_accuracies)
overall_precision = sum(precisions) / len(precisions)
overall_recall = sum(recalls) / len(recalls)
overall_f1_score = sum(f1_scores) / len(f1_scores)

print(f"Overall Average Accuracy: {overall_average_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1-Score: {overall_f1_score}")

Train Accuracy for I/E: 0.9164265129682997
Test Accuracy for I/E: 0.7567723342939481
Average Accuracy for I/E: 0.8365994236311238
Precision for I/E: 0.769185360094451
Recall for I/E: 0.9767616191904048
F1-Score for I/E: 0.8606340819022458

Train Accuracy for N/S: 0.9118155619596542
Test Accuracy for N/S: 0.861671469740634
Average Accuracy for N/S: 0.8867435158501441
Precision for N/S: 0.861671469740634
Recall for N/S: 1.0
F1-Score for N/S: 0.9256965944272446

Train Accuracy for F/T: 0.9289625360230548
Test Accuracy for F/T: 0.7118155619596542
Average Accuracy for F/T: 0.8203890489913546
Precision for F/T: 0.6887360275150473
Recall for F/T: 0.853035143769968
F1-Score for F/T: 0.7621313035204568

Train Accuracy for J/P: 0.9182997118155619
Test Accuracy for J/P: 0.5971181556195966
Average Accuracy for J/P: 0.7577089337175793
Precision for J/P: 0.48770491803278687
Recall for J/P: 0.34643377001455605
F1-Score for J/P: 0.4051063829787234

Overall Average Accuracy: 0.8253602305475505
Overall 

In [ ]:
# Define the SVM model
svm_model = SVC(random_state=42)

In [ ]:
# Train the model on class type
svm_model.fit(train_combined_features, train_df['type'])

SVC(random_state=42)

In [ ]:
# Predict on the train set and map to dimensions
train_predictions = svm_model.predict(train_combined_features)
train_pred_dimensions = pd.DataFrame([get_mbti_dimensions(pred) for pred in train_predictions], columns=['I/E', 'N/S', 'F/T', 'J/P'])

In [ ]:
# Predict on the test set and map to dimensions
test_predictions = svm_model.predict(test_combined_features)
test_pred_dimensions = pd.DataFrame([get_mbti_dimensions(pred) for pred in test_predictions], columns=['I/E', 'N/S', 'F/T', 'J/P'])

In [ ]:
# Initialize lists to store metrics for each dimension
average_accuracies = []
precisions = []
recalls = []
f1_scores = []

# Calculate metrics for each dimension and their averages
for dimension in ['I/E', 'N/S', 'F/T', 'J/P']:
    train_accuracy = accuracy_score(train_df[dimension], train_pred_dimensions[dimension])
    test_accuracy = accuracy_score(test_df[dimension], test_pred_dimensions[dimension])

    # Calculate the average accuracy for this dimension
    average_accuracy = (train_accuracy + test_accuracy) / 2
    average_accuracies.append(average_accuracy)

    # Calculate precision, recall, and f1-score for the test set
    precision = precision_score(test_df[dimension], test_pred_dimensions[dimension])
    recall = recall_score(test_df[dimension], test_pred_dimensions[dimension])
    f1 = f1_score(test_df[dimension], test_pred_dimensions[dimension])

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    print(f"Train Accuracy for {dimension}: {train_accuracy}")
    print(f"Test Accuracy for {dimension}: {test_accuracy}")
    print(f"Average Accuracy for {dimension}: {average_accuracy}")
    print(f"Precision for {dimension}: {precision}")
    print(f"Recall for {dimension}: {recall}")
    print(f"F1-Score for {dimension}: {f1}\n")

# If you want to calculate an overall average accuracy, precision, recall, and f1-score across all dimensions
overall_average_accuracy = sum(average_accuracies) / len(average_accuracies)
overall_precision = sum(precisions) / len(precisions)
overall_recall = sum(recalls) / len(recalls)
overall_f1_score = sum(f1_scores) / len(f1_scores)

print(f"Overall Average Accuracy: {overall_average_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1-Score: {overall_f1_score}")

Train Accuracy for I/E: 0.776657060518732
Test Accuracy for I/E: 0.7659942363112392
Average Accuracy for I/E: 0.7713256484149855
Precision for I/E: 0.7703962703962703
Recall for I/E: 0.9910044977511244
F1-Score for I/E: 0.8668852459016394

Train Accuracy for N/S: 0.8622478386167147
Test Accuracy for N/S: 0.861671469740634
Average Accuracy for N/S: 0.8619596541786744
Precision for N/S: 0.861671469740634
Recall for N/S: 1.0
F1-Score for N/S: 0.9256965944272446

Train Accuracy for F/T: 0.776657060518732
Test Accuracy for F/T: 0.7446685878962536
Average Accuracy for F/T: 0.7606628242074928
Precision for F/T: 0.7271062271062271
Recall for F/T: 0.845580404685836
F1-Score for F/T: 0.7818808468734613

Train Accuracy for J/P: 0.6779538904899135
Test Accuracy for J/P: 0.6478386167146974
Average Accuracy for J/P: 0.6628962536023055
Precision for J/P: 0.5811965811965812
Recall for J/P: 0.39592430858806404
F1-Score for J/P: 0.47099567099567097

Overall Average Accuracy: 0.7642110951008645
Overall P

In [ ]:
# Define the LightGBM model
lgbm_model = LGBMClassifier(random_state=42)

In [ ]:
# Train the model on class type
lgbm_model.fit(train_combined_features, train_df['type'])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055901 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 203990
[LightGBM] [Info] Number of data points in the train set: 6940, number of used features: 800
[LightGBM] [Info] Start training from score -3.821177
[LightGBM] [Info] Start training from score -2.553488
[LightGBM] [Info] Start training from score -3.624701
[LightGBM] [Info] Start training from score -2.538782
[LightGBM] [Info] Start training from score -5.348549
[LightGBM] [Info] Start training from score -5.207471
[LightGBM] [Info] Start training from score -5.411070
[LightGBM] [Info] Start training from score -4.582377
[LightGBM] [Info] Start training from score -1.775183
[LightGBM] [Info] Start training from score -1.554764
[LightGBM] [Info] Start training from score -2.073121
[LightGBM] [Info] Start training from score -1.895201
[LightGBM] [Info] Start training from score -3.954708
[LightG

LGBMClassifier(random_state=42)

In [ ]:
# Predict on the train set and map to dimensions
train_predictions = lgbm_model.predict(train_combined_features)
train_pred_dimensions = pd.DataFrame([get_mbti_dimensions(pred) for pred in train_predictions], columns=['I/E', 'N/S', 'F/T', 'J/P'])

In [ ]:
# Predict on the test set and map to dimensions
test_predictions = lgbm_model.predict(test_combined_features)
test_pred_dimensions = pd.DataFrame([get_mbti_dimensions(pred) for pred in test_predictions], columns=['I/E', 'N/S', 'F/T', 'J/P'])

In [ ]:
# Initialize lists to store metrics for each dimension
average_accuracies = []
precisions = []
recalls = []
f1_scores = []

# Calculate metrics for each dimension and their averages
for dimension in ['I/E', 'N/S', 'F/T', 'J/P']:
    train_accuracy = accuracy_score(train_df[dimension], train_pred_dimensions[dimension])
    test_accuracy = accuracy_score(test_df[dimension], test_pred_dimensions[dimension])

    # Calculate the average accuracy for this dimension
    average_accuracy = (train_accuracy + test_accuracy) / 2
    average_accuracies.append(average_accuracy)

    # Calculate precision, recall, and f1-score for the test set
    precision = precision_score(test_df[dimension], test_pred_dimensions[dimension])
    recall = recall_score(test_df[dimension], test_pred_dimensions[dimension])
    f1 = f1_score(test_df[dimension], test_pred_dimensions[dimension])

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    print(f"Train Accuracy for {dimension}: {train_accuracy}")
    print(f"Test Accuracy for {dimension}: {test_accuracy}")
    print(f"Average Accuracy for {dimension}: {average_accuracy}")
    print(f"Precision for {dimension}: {precision}")
    print(f"Recall for {dimension}: {recall}")
    print(f"F1-Score for {dimension}: {f1}\n")

# If you want to calculate an overall average accuracy, precision, recall, and f1-score across all dimensions
overall_average_accuracy = sum(average_accuracies) / len(average_accuracies)
overall_precision = sum(precisions) / len(precisions)
overall_recall = sum(recalls) / len(recalls)
overall_f1_score = sum(f1_scores) / len(f1_scores)

print(f"Overall Average Accuracy: {overall_average_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1-Score: {overall_f1_score}")

Train Accuracy for I/E: 1.0
Test Accuracy for I/E: 0.7729106628242075
Average Accuracy for I/E: 0.8864553314121038
Precision for I/E: 0.7919254658385093
Recall for I/E: 0.9557721139430285
F1-Score for I/E: 0.8661684782608695

Train Accuracy for N/S: 1.0
Test Accuracy for N/S: 0.8645533141210374
Average Accuracy for N/S: 0.9322766570605188
Precision for N/S: 0.8658536585365854
Recall for N/S: 0.9973244147157191
F1-Score for N/S: 0.9269505750699409

Train Accuracy for F/T: 1.0
Test Accuracy for F/T: 0.7556195965417868
Average Accuracy for F/T: 0.8778097982708934
Precision for F/T: 0.7426955702167767
Recall for F/T: 0.8391906283280085
F1-Score for F/T: 0.788

Train Accuracy for J/P: 1.0
Test Accuracy for J/P: 0.6426512968299711
Average Accuracy for J/P: 0.8213256484149856
Precision for J/P: 0.5557404326123128
Recall for J/P: 0.4861717612809316
F1-Score for J/P: 0.5186335403726708

Overall Average Accuracy: 0.8794668587896254
Overall Precision: 0.7390537818010461
Overall Recall: 0.81961472